In [79]:
from colorama import init
init()
from colorama import Fore, Back, Style

In [80]:
alph = "АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ"
# alph = "абвгдежзийклмнопрстуфхцчшщъыьэюя"
# alph = "АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюя"

len_alph = len(alph)

In [81]:
class Playfair:
    def __init__(self, breakdown: str = 'Х', matrix_size: str = (4, 8)):
        self.breakdown = breakdown
        self.matrix: list[str] = list()
        self.matrix_size = matrix_size

    def set_matrix(self, phrase: str) -> None:
        lines_for_phrase = len(phrase) // self.matrix_size[0]
        ost_phrase = phrase[len(phrase) - 1 - (len(phrase) % self.matrix_size[0]):]
        for i in range(lines_for_phrase):
            self.matrix.append(phrase[i*self.matrix_size[0]:(i+1)*self.matrix_size[0]])
        phrase_set = set(phrase)
        ost_alph = "".join([i for i in alph if i not in phrase_set])
        ost = ost_phrase + ost_alph
        ost_lines = self.matrix_size[1] - lines_for_phrase
        for i in range(ost_lines):
            self.matrix.append(ost[i*self.matrix_size[0]:(i+1)*self.matrix_size[0]])
        
    def print_matrix(
        self,
        marked: list[list[tuple[int, int]]] | None = None,
        file = None
    ) -> None:
        matrix_to_print = []
        for line in self.matrix:
            matrix_to_print.append([f" {sym} " for sym in line])

        if marked is not None:
            for i, marked_group in enumerate(marked):
                for pixel in marked_group:
                    if i == 0:
                        sym = f"({self.matrix[pixel[0]][pixel[1]]})"
                    else:
                        sym = f"[{self.matrix[pixel[0]][pixel[1]]}]"
                    matrix_to_print[pixel[0]][pixel[1]] = sym

        for line in matrix_to_print:
            print(*line, sep=" ", file=file)

    def prepare_data_to_encrypt(self, data: str) -> list[str]:
        result = []
        last = None
        for i in data:
            if last is None:
                last = i
                continue

            if last == i:
                result.append(last + self.breakdown)
                last = self.breakdown
                continue
                
            result.append(last + i)
            last = None
        
        if last is not None:
            result.append(last + self.breakdown)
        
        return result
    
    def _encrypt_bigram(self, bi: str) -> str:
        bi1 = [None, None]
        bi2 = [None, None]

        for i, line in zip(range(len(self.matrix)), self.matrix):
            if (index_1 := line.find(bi[0])) != -1:
                bi1 = [i, index_1]
                break
        else:
            raise Exception("NOT FOUND IN MATRIX")

        for i, line in zip(range(len(self.matrix)), self.matrix):
            if (index_1 := line.find(bi[1])) != -1:
                bi2 = [i, index_1]
                break
        else:
            raise Exception("NOT FOUND IN MATRIX")

        if bi1[0] == bi2[0]:
            bi1[1] = (bi1[1] + 1) % self.matrix_size[0]
            bi2[1] = (bi2[1] + 1) % self.matrix_size[0]
        elif bi1[1] == bi2[1]:
            bi1[0] = (bi1[0] + 1) % self.matrix_size[1]
            bi2[0] = (bi2[0] + 1) % self.matrix_size[1]
        else:
            bi1, bi2 = (bi2[0], bi1[1]), (bi1[0], bi2[1])
        return self.matrix[bi1[0]][bi1[1]] + self.matrix[bi2[0]][bi2[1]]
    
    def encrypt(self, data: str) -> str:
        if not self.matrix:
            raise Exception("NO DATA IN MATRIX")
        prepared_data = self.prepare_data_to_encrypt(data)
        encrypted_data = ""
        for bigram in prepared_data:
            encrypt_bigram = self._encrypt_bigram(bigram)
            encrypted_data += encrypt_bigram
        return encrypted_data
    

    def _visual_encrypting_bigram(self, plain_bi: str) -> str:
        print(f"шифруем биграмму {plain_bi}")
        plain_bi1 = [None, None]
        plain_bi2 = [None, None]

        for i, line in zip(range(len(self.matrix)), self.matrix):
            if (index_1 := line.find(plain_bi[0])) != -1:
                plain_bi1 = [i, index_1]
                break
        else:
            raise Exception("NOT FOUND IN MATRIX")

        for i, line in zip(range(len(self.matrix)), self.matrix):
            if (index_1 := line.find(plain_bi[1])) != -1:
                plain_bi2 = [i, index_1]
                break
        else:
            raise Exception("NOT FOUND IN MATRIX")
        
        crypted_bi = None
        crypted_bi1 = plain_bi1.copy()
        crypted_bi2 = plain_bi2.copy()

        if plain_bi1[0] == plain_bi2[0]:
            crypted_bi1[1] = (plain_bi1[1] + 1) % self.matrix_size[0]
            crypted_bi2[1] = (plain_bi2[1] + 1) % self.matrix_size[0]
        elif plain_bi1[1] == plain_bi2[1]:
            crypted_bi1[0] = (plain_bi1[0] + 1) % self.matrix_size[1]
            crypted_bi2[0] = (plain_bi2[0] + 1) % self.matrix_size[1]
        else:
            crypted_bi1, crypted_bi2 = (plain_bi2[0], plain_bi1[1]), (plain_bi1[0], plain_bi2[1])
        crypted_bi = self.matrix[crypted_bi1[0]][crypted_bi1[1]] + self.matrix[crypted_bi2[0]][crypted_bi2[1]]
        
        self.print_matrix(marked = [[plain_bi1, plain_bi2], [crypted_bi1, crypted_bi2]])

        print(f"из {plain_bi} получаем {crypted_bi}")
        print("----------")
        return crypted_bi
    

    def print_visual(self, data: str):
        if not self.matrix:
            raise Exception("NO DATA IN MATRIX")
        prepared_data = self.prepare_data_to_encrypt(data)
        for bigram in prepared_data:
            self._visual_encrypting_bigram(bigram)




In [82]:
# Демонстрация работы
key = "КЛЮЧ"
data = "ПРИВЕТМИРБУКАБЯКАЧТОТО"

print(f"Исходный текст: {data}")
print(f"Ключ: {key}\n")


playfair_cipher = Playfair()
playfair_cipher.set_matrix("РАНДОМНАЯФРАЗА")

print("шифр Плейфера:")
print(f"{key} - ключ")
print(f"{data} - исходное")
print(f"{playfair_cipher.encrypt(data)} - зашифровано")
print()

Исходный текст: ПРИВЕТМИРБУКАБЯКАЧТОТО
Ключ: КЛЮЧ

шифр Плейфера:
КЛЮЧ - ключ
ПРИВЕТМИРБУКАБЯКАЧТОТО - исходное
ХОПИКЧЙОАДЛТЗДИРЦННПНП - зашифровано



In [83]:
playfair_cipher.print_visual(data)

шифруем биграмму ПР
(Р)  А   Н   Д 
[О]  М   Н   А 
 Я   Ф   Р   А 
 А   З   А   Б 
 В   Г   Е   Ж 
 И   Й   К   Л 
(П)  С   Т   У 
[Х]  Ц   Ч   Ш 
из ПР получаем ХО
----------
шифруем биграмму ИВ
 Р   А   Н   Д 
 О   М   Н   А 
 Я   Ф   Р   А 
 А   З   А   Б 
(В)  Г   Е   Ж 
[И]  Й   К   Л 
[П]  С   Т   У 
 Х   Ц   Ч   Ш 
из ИВ получаем ПИ
----------
шифруем биграмму ЕТ
 Р   А   Н   Д 
 О   М   Н   А 
 Я   Ф   Р   А 
 А   З   А   Б 
 В   Г  (Е)  Ж 
 И   Й  [К]  Л 
 П   С  (Т)  У 
 Х   Ц  [Ч]  Ш 
из ЕТ получаем КЧ
----------
шифруем биграмму МИ
 Р   А   Н   Д 
[О] (М)  Н   А 
 Я   Ф   Р   А 
 А   З   А   Б 
 В   Г   Е   Ж 
(И) [Й]  К   Л 
 П   С   Т   У 
 Х   Ц   Ч   Ш 
из МИ получаем ЙО
----------
шифруем биграмму РБ
(Р)  А   Н  [Д]
 О   М   Н   А 
 Я   Ф   Р   А 
[А]  З   А  (Б)
 В   Г   Е   Ж 
 И   Й   К   Л 
 П   С   Т   У 
 Х   Ц   Ч   Ш 
из РБ получаем АД
----------
шифруем биграмму УК
 Р   А   Н   Д 
 О   М   Н   А 
 Я   Ф   Р   А 
 А   З   А   Б 
 В   Г   Е   Ж 
 И   Й  (К) [Л]